In [1]:
DATA_NAME = 'syntheticMRI2D-sagittal' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_horizontal_wavelet_sagittal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,2136778.81,5.47,8.27,-8.70,-0.10,-0.70,-0.07
4,5.47,168481.18,-3.22,-2.23,1.66,0.30,0.10
5,8.27,-3.22,83695.71,0.34,0.78,0.73,-0.12
6,-8.70,-2.23,0.34,26501.01,-0.05,-0.02,-0.04
7,-0.10,1.66,0.78,-0.05,6868.02,0.06,0.03
8,-0.70,0.30,0.73,-0.02,0.06,977.01,0.01
9,-0.07,0.10,-0.12,-0.04,0.03,0.01,94.97


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,0.00001,0.00002,-0.00004,-0.00000,-0.00002,-0.00001
4,0.00001,1.00000,-0.00003,-0.00003,0.00005,0.00002,0.00002
5,0.00002,-0.00003,1.00000,0.00001,0.00003,0.00008,-0.00004
6,-0.00004,-0.00003,0.00001,1.00000,-0.00000,-0.00000,-0.00003
7,-0.00000,0.00005,0.00003,-0.00000,1.00000,0.00002,0.00004
8,-0.00002,0.00002,0.00008,-0.00000,0.00002,1.00000,0.00003
9,-0.00001,0.00002,-0.00004,-0.00003,0.00004,0.00003,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

19.696237968851122

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.52404095e+11 3.99638460e+09 9.29549484e+08 8.77861750e+07
 5.22292264e+06 7.98689641e+04 2.07206486e-27]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.999889,-0.013212,-0.006525,-0.002068,-0.000535,-0.000076,-0.000007
1,0.012198,0.991241,-0.127920,-0.029543,-0.007434,-0.001056,-0.000102
2,0.007890,0.124480,0.987729,-0.091583,-0.020912,-0.002943,-0.000288
3,0.003077,0.039729,0.084834,0.991071,-0.094044,-0.012203,-0.001186
4,0.001071,0.013589,0.027455,0.090385,0.992688,-0.073575,-0.006933
5,0.000226,0.002864,0.005760,0.018235,0.071021,0.992507,-0.097524
6,0.000044,0.000560,0.001130,0.003568,0.013756,0.096731,0.995208


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.000110861320950284,
 0.008759130770238932,
 0.01227085491043478,
 0.008929495592502334,
 0.007311951432163455,
 0.007492536621851231,
 0.004791748287006126]